# Práctica 1. Explorando resultados de RNA-SEQ. MASTER BIOINFORMÁTICA
##### Realizada por Fernando Lucas Ruiz




## 1. Importar librerias

En primer lugar, para una correcta visualización, ejecutar los bloques de código en orden 

Las librerias que he utilizado para esta práctica son:
- __zipfile__ para descargar los archivos zip
- __csv__ para cargar los archivos csv
- __Path__ de pathlib para manipular rutas
- __re__ para generar expresiones regulares a la hora de manipular errores o buscar ficheros
- __math__ para realizar los cálculos logarítmicos


In [1]:
import zipfile
import csv
from pathlib import Path
import re
import math

## 2. Funciones
### Funcion load_runs
Con esta funcion pretendo abrir los archivos de las distintas carpetas. 
1. Descomprimir el archivo .zip que contienen los RUN.

2. Para ello lo que hago es definir la ruta y de esa ruta buscar los directorios que contengan el nombre RUN mediante un bucle. Genero la expresion regular para sacar el nombre y numero del RUN.

3. Uno la ruta base al subdirectorio para abrir el archivo csv mediante un bucle

4. Este bucle está dividido en dos partes. La primera es para definir el limitidar del archivo CSV abriendo el archivo mediante reader y buscando en la primera fila. La segunda parte abro esta vez el run mediante DictReader y lo guardo en un diccionario para devolver al final de la función.


La estructura del diccionario_resultado es:
- {NombreRUN: [{GENE: NombreGen, NombreMuestra: expresion,...}]}

In [2]:
def load_runs(ruta):
    base_path = Path(ruta) # ruta en la que se encuentra el archivo .zip
    path_zipfile = base_path.joinpath("runs.zip")

    with zipfile.ZipFile(path_zipfile, "r") as filezip:
        filezip.extractall(base_path)
        
        # Buscar y nombrar los directorios RUN
        diccionario_resultado = {}
        for subfolder in base_path.rglob("RUN*"):
            er=re.compile("(RUN)+[0-9]")
            run = er.search(str(subfolder)).group()

            file_path = base_path.joinpath(subfolder)

            # Búsqueda de delimiter
            for file in file_path.rglob("experimento*"):
                with open(file, mode = "r") as csv_file:
                    csv_reader = csv.reader(csv_file)
                    contador = 0
                    for row in csv_reader:
                            if contador == 0:
                                if ";" in row[0]:
                                    delimitador = ";"
                                    contador +=1
                                    continue
                                elif "," in row[0]:
                                    delimitador = ";"
                                    contador +=1
                                    continue
                                elif "|" in row[0]:
                                    delimitador = ";"
                                    contador +=1
                                    continue
                                elif "#" in row[0]:
                                    delimitador = ";"
                                    contador +=1
                                    continue
                                else:
                                    delimitador = '\t'
                                    contador +=1
                            else:
                                break

                    # Guardar los datos en diccionario
                    with open(file, mode = "r") as csv_file:
                            csv_reader = csv.DictReader(csv_file, delimiter = delimitador)
                            lista_resultado = []
                            for row in csv_reader:
                                lista_resultado.append(row)

            diccionario_resultado[run] = lista_resultado
    return diccionario_resultado

### Función informacion_general
Con el argumento estructura_datos_runs proveniente de la funcion load_runs, genero un bucle para extraer de cada RUN su nombre, la lista de genes para calcular el numero de genes que tiene ese RUN y el numero de muestras WT y KO mediante contadores. Todo ello lo meto a un diccionario llamado diccionario_run_total. 
Por otro lado de cada run voy sumando los numeros de WT y KO para luego cuantificar el numero total de cada muestra, además, voy metiendo los genes en una lista de genes de todos los run para ver si entre RUN hay genes diferentes.

La estructura del diccionario_run_total es:
- {NombreRUN: {'WT': numero de wild types, 'KO': Numero de KOs, 'Numero_genes': numero de genes}...}

Con esta función resuelvo el ejercicio 1.


In [3]:
def informacion_general(estructura_datos_runs):

    diccionario_run_total = {}
    contador_total_wt = 0
    contador_total_ko = 0
    lista_total_genes = []

    for run in estructura_datos_runs:
        lista_run_genes = []
        diccionario_run = {}
        for datos_run in estructura_datos_runs[run]:
            contador_run_wt = 0
            contador_run_ko = 0
            for muestra in datos_run:
                if muestra == "GENE":
                    gen = datos_run[muestra]
                    if gen not in lista_run_genes:
                        lista_run_genes.append(gen)
                    if gen not in lista_total_genes:
                        lista_total_genes.append(gen)
                    continue
                elif muestra.startswith("WT"):
                    contador_run_wt += 1
                elif muestra.startswith("KO"):
                    contador_run_ko +=1
                
            diccionario_run["WT"]=contador_run_wt
            diccionario_run["KO"]=contador_run_ko
            diccionario_run["Numero_genes"] = len(lista_run_genes)
        contador_total_wt +=contador_run_wt
        contador_total_ko += contador_run_ko

        diccionario_run_total[run] = diccionario_run

    return diccionario_run_total, contador_total_wt, contador_total_ko, lista_total_genes

### Función inf_detallada_run
Con los argumentos estructura_datos_runs proveniente de la funcion load_runs, el nombre del run y la selección del usuario para elegir entre las 6 opciones previstas.
1. Según la selección del usuario (entre el 1 y el 6), nombro a la muestra. Según las opciones son el 1 y 4 para todas las muestras, 2 y 5 para ver solo WT, y 3 y 6 para ver los KO.

2. Llamo a la funcion anterior informacion_general para extraer los num_wt y num_ko para calcular las medias.

3. Me genero varios diccionarios para extraer la información de estructura_datos_runs y corro un bucle para ir sumando los valores de cada muestra de cada gen para al final calcular su media y meterla en 3 diccionarios distintos (diccionario_genes_expresion_total, diccionario_genes_expresion_wt y diccionario_genes_expresion_wt).

4. Despues del bucle meto cada diccionario en el diccionario_genes_expresion en 3 keys diferentes (Total, WT, KO)

5. Genero un if-else para, según haya pedido el usuario, sacar los genes más expresados (selección usuario <= 3) o menos expresados (selección usuario >= 3). Ambos bucles generan los 10 primeros valores del diccionario ordenado según las medias guardadas en diccionario_genes_expresion.

6. Finalmente devuelvo de la función los datos del diccionario_genes_expresión para utilizarlo en la función log_fold_change(más abajo), los 10 genes sobre- o infra- expresados del run y el nombre de la muestra elegida por el usuario según si es el Total, WT o KO

In [4]:
def inf_detallada_run(estructura_datos_runs, run, seleccion_usuario):

    # Nombrar la selección del usuario a total, wt o ko
    if seleccion_usuario == 1 or seleccion_usuario == 4:
        selecion_usuario_muestra = "Total"
    elif seleccion_usuario == 2 or seleccion_usuario == 5:
        selecion_usuario_muestra = "WT"
    elif seleccion_usuario == 3 or seleccion_usuario == 6:
        selecion_usuario_muestra = "KO"

    # Extraer diccionario run total para coger num_wt num_ko
    igr = informacion_general(estructura_datos_runs)
    diccionario_run_total = igr[0]
    for muestra, value in diccionario_run_total[run].items():
        if muestra.startswith("WT"):
            num_wt = value
        if muestra.startswith("KO"):
            num_ko = value

    # Extraer expresion media de los genes y meterlos en el diccionario_genes_expresion
    diccionario_genes_expresion = {}
    diccionario_genes_expresion_total = {}
    diccionario_genes_expresion_wt = {}
    diccionario_genes_expresion_ko = {}
    for datos in estructura_datos_runs[run]:
        nombre_gen = datos["GENE"]
        suma_expresion_total = 0
        suma_expresion_wt = 0
        suma_expresion_ko = 0

        for muestra in datos.keys():
            if muestra == "GENE":
                continue
            if muestra.startswith("WT"):
                suma_expresion_wt += int(datos[muestra])
            elif muestra.startswith("KO"):
                suma_expresion_ko += int(datos[muestra])
            suma_expresion_total += int(datos[muestra])

        diccionario_genes_expresion_total[nombre_gen] = suma_expresion_total / (num_wt + num_ko)
        diccionario_genes_expresion_wt[nombre_gen] = suma_expresion_wt/num_wt
        diccionario_genes_expresion_ko[nombre_gen] = suma_expresion_ko / num_ko

    diccionario_genes_expresion["Total"] = diccionario_genes_expresion_total
    diccionario_genes_expresion["WT"] = diccionario_genes_expresion_wt
    diccionario_genes_expresion["KO"] = diccionario_genes_expresion_ko

    # Extraer top10 genes mas o menos expresados
    if seleccion_usuario <= 3 :
        mas_expresados=dict(sorted(diccionario_genes_expresion[selecion_usuario_muestra].items(), key=lambda x:x[1], reverse=True))
        index = 0
        top_10 = {}
        for key, value in mas_expresados.items():
            if index < 10:
                top_10[key] = value
                index += 1
    elif seleccion_usuario >= 4:
        menos_expresados = dict(sorted(diccionario_genes_expresion[selecion_usuario_muestra].items(), key=lambda x:x[1]))
        index = 0
        top_10 = {}
        for key, value in menos_expresados.items():
            if index < 10:
                top_10[key] = value
                index += 1
        
    return diccionario_genes_expresion, top_10, selecion_usuario_muestra

### Función log_fold_change
Con los argumentos diccionario_genes_expresion de la funcion inf_detallada_run, y la seleccion del valor float elegida por el usuario.

1. Para el primer bucle calculo el log fold change comparando la muestra experimental (KO) contra la muestra control(WT) y lo meto al diccionario_genes_expresion_log2.

2. Para el segundo bucle, cojo ese diccionario recien creado y, según el valor absoluto de la seleccion del log fold change del usuario, meter en un diccionario los más expresados y en otro los menos expresados.

3. Finalmente, genero un diccionario_lfc_usuario con las muestras más y menos reguladas según el punto de corte del usuario y la devuelvo

In [5]:
def log_fold_change(diccionario_genes_expresion, selecion_usuario_lfc):
    
    # Calcular log fold change de cada gen 
    diccionario_genes_expresion_log2 = {}
    for muestra in diccionario_genes_expresion:
        if muestra == "WT" :
            for gene,value in diccionario_genes_expresion[muestra].items() :
                if gene in diccionario_genes_expresion["KO"]:
                    exp_ko = diccionario_genes_expresion["KO"][gene]
                    log_base2 = math.log2(exp_ko / value)
                    diccionario_genes_expresion_log2[gene] = log_base2
        if muestra == "KO":
            break

    # Meter los genes con mas fold change segun corte usuario
    diccionario_lfc_usuario = {}
    diccionario_lfc_usuario_upregulated = {}
    diccionario_lfc_usuario_downregulated = {}
    for gene, value in diccionario_genes_expresion_log2.items():
        if abs(value) >= selecion_usuario_lfc:
            if value > 0 :
                diccionario_lfc_usuario_upregulated[gene] = round(value,3) #redondeo a 3 decimales para que sea más claro
            if value < 0 :
                diccionario_lfc_usuario_downregulated[gene] = round(value,3) #redondeo a 3 decimales para que sea más claro

    diccionario_lfc_usuario["Genes sobreexpresados"] = diccionario_lfc_usuario_upregulated
    diccionario_lfc_usuario["Genes infraexpresados"] = diccionario_lfc_usuario_downregulated

    return diccionario_lfc_usuario

### Funciones de error
Defino tres funciones de error, para controlar que el usuario mete correctamente los valores que quiero que introduzca en cada input. 

1. El primero es para asegurarme de que mete bien el numero de run. Meto longitud_runs por si en algun momento metemos más RUN en un futuro.

2. El segundo para la selección del numero de la seleccion detallada de los genes más o menos expresados en cada RUN.

3. Para controlar la selección correcta del numero log fold change.

En todos ellos pongo la selección de m para volver al menú inicial.

In [6]:
def error_run(numero_run, longitud_runs):
    while True:
        er=re.compile("[a-zA-Z]")

        if numero_run == "m":
            main()
            break

        elif er.search(numero_run) != None:
            print("\n","\n","ERROR: Debe ser un numero entero. Intentalo de nuevo.", 
                  "\n","Para volver al menu inicial pulsa m.")
            numero_run = input("Inserta de nuevo otro numero")

        elif int(numero_run) > longitud_runs:
            print("\n","\n","ERROR: Ese numero de RUN no esta cargado. Intentalo de nuevo.", 
                  "\n","Para volver al menu inicial pulsa m.")
            numero_run = input("Inserta de nuevo otro numero")

        elif int(numero_run) <= 0:
            print("\n","\n","ERROR: Debe ser un numero positivo. Intentalo de nuevo.", 
                  "\n","Para volver al menu inicial pulsa m.")
            numero_run = input("Inserta de nuevo otro numero")
            
        else:
            return numero_run
        
def error_usuario(selecion_usuario):
    while True:
            
        er=re.compile("[a-zA-Z]")

        if selecion_usuario == "m":
            main()
            break

        elif er.search(selecion_usuario) != None:
            print("\n\n","ERROR: Debe ser un numero entero. Intentalo de nuevo.", 
                  "\n","Para volver al menu inicial pulsa m.")
            selecion_usuario = input("Inserta de nuevo otro numero")

        elif int(selecion_usuario) >= 7 or int(selecion_usuario) <= 0:
            print("\n\n","ERROR: Elige entre las siguientes opciones. Intentalo de nuevo.", 
                  "\n","Para volver al menu inicial pulsa m.")
            print("\n\t","Escriba la selección que desee:","\n",
                    "\n\t\t", "1. Los 10 genes más expresados en su valor medio en todas las muestras",
                    "\n\t\t", "2. Los 10 genes más expresados en su valor medio en todas las muestras WT",
                    "\n\t\t", "3. Los 10 genes más expresados en su valor medio en todas las muestras KO",
                    "\n\t\t", "4. Los 10 genes menos expresados en su valor medio en todas las muestras",
                    "\n\t\t","5. Los 10 genes menos expresados en su valor medio en todas las muestras WT", 
                    "\n\t\t", "6. Los 10 genes menos expresados en su valor medio en todas las muestras KO",
                    "\n\n")
            selecion_usuario = input("Inserta de nuevo el numero")

        else: 
            return selecion_usuario

def error_lfc(selecion_usuario_lfc):
    while True: 
        er=re.compile("[a-zA-Z]")

        if selecion_usuario_lfc == "m":
            main()
            break
        if er.search(selecion_usuario_lfc) != None:
                print("\n\n","ERROR: Debe ser un numero. Intentalo de nuevo.", 
                    "\n","Para volver al menu inicial pulsa m.")
                selecion_usuario_lfc = input("Inserta de nuevo otro numero")

        elif float(selecion_usuario_lfc) <= 0 :
            print("\n\n","ERROR: Debe ser un numero positivo. Intentalo de nuevo.", 
                    "\n","Para volver al menu inicial pulsa m.")
            selecion_usuario_lfc = input("Inserta de nuevo otro numero")

        else: 

            return selecion_usuario_lfc

## 3. Main
1. Cargo y ordeno los datos segun el numero del RUN

2. Se lanza por pantalla la selección que quiera el usuario.

3. Para la opción 1:

     1. Le muestro al usario los RUN que están cargados y la información en cada RUN del numero de muestras y el numero de genes.

     2. Le doy la opción al usuario a volver al menú o elegir el RUN que quiera para ver la información más en detalle, dando la opcion de volver al menú inicial

     3. Según lo que elija el usuario lo paso por la funcion error_usuario para ver si lo introduce correctamente

     4. Si es correcto, muestro por pantalla las 6 opciones para ver los genes mas o menos, según total o de cada muestra del RUN elegido, dando la opcion de volver al menú inicial.

     5. Según lo que elija el usuario lo paso por la funcion error_usuario para ver si lo introduce correctamente.

     6. Si es correcto, según la elección del usuario, muestro los top 10 genes más o menos expresados, según total o de cada muestra del RUN elegido.

     7. También saco por pantalla si quiere ver el log fold change de los genes de ese RUN, permitiendo al usuario que elija el punto de corte del log fold change, y dando la opcion de volver al menú inicial.

     8.  Según lo que elija el usuario lo paso por la funcion error_lfc para ver si lo introduce correctamente. Despues le paso un Try-except para ver si ha introducido un valor float.

     9. Sacó por pantalla los genes más expresados (log fold change positivo) y los menos expresados (log fold change negativo) según el corte del usuario, en orden descendente.

     10. Finalmente le pregunto al usuario si quiere volver al menu incial o acabar el programa

4. Para la opción 2:

     1. Cargo los datos de la informacion_general y muestro por pantalla los RUN cargados, el numero total de muestras y los genes de todos los RUN.
     
     2. Finalmente le pregunto al usuario si quiere volver al menu incial o acabar el programa.

In [7]:
def main():
    # descarga los RUN
    estructura_datos_runs = load_runs("/Users/fernandolucasruiz/Library/CloudStorage/OneDrive-UNIVERSIDADDEMURCIA/Documentos/Fernando/Master Bioinformatica/Asignaturas/Sistemas Bioinformáticos/Bloque3_programacion/Tareas 2023/zipruns/")
    estructura_datos_runs = dict(sorted(estructura_datos_runs.items(), key=lambda x:x[0]))
    #Entrada del usuario
    while True:
        print("\n\n1. Informacion de los RUNs")
        print("2. Total de muestras cargadas")
        print("3. Salir del programa")
        opcion = input("\nIndique el opción que desea ejecutar (1, 2 o 3): ")

        
        if opcion == "1":
            while True:
                print("\n","\n","\t","Has marcado la opcion 1. Por favor indica el RUN para ver la información general.",
                      "\n","\t","Para volver al menu inicial pulsa m", 
                      "\n","\n","\t","Actualmente tenemos cargados los siguientes RUN:")
                # Muestro los RUN que están cargados
                for run in estructura_datos_runs:
                    print("\t",run)
                longitud_runs = len(estructura_datos_runs) # guardo numero de RUN que están cargados

                # De la función informacion_general guardo todos los datos en igr para luego sacar el diccionario con los datos de num_wt, num_ko y numero de genes por RUN.
                igr = informacion_general(estructura_datos_runs)
                diccionario_run_total = igr[0]

                for run in diccionario_run_total:
                    print("\n\t","Los datos del", run, "son:")
                    for muestra, value in diccionario_run_total[run].items():
                        if muestra == "Numero_genes":
                            print("\t\t", "Con un total de", value, "genes.")

                        else:
                            print("\t\t", "El numero de muestras", muestra, "del", run, "es: ", value,)
                

                # Lanzo otra seleccion del usuario para que elija el RUN que quiera ver más en detalle
                print("\n","\n","\t","Para ver más detalle de un RUN en concreto, por favor indica el RUN para ver la información detallada.",
                      "\n","\t","Para volver al menu inicial pulsa m") 
                
                numero_run = input("\nEscriba el numero del RUN que quiere ver los datos.")

                numero_run = error_run(numero_run,longitud_runs) #filtro por si hubiese error del usuario al marcar RUN
                
                if numero_run == None:
                    break

                run = "RUN"+str(numero_run)

                # Elección del usuario entre las 6 opciones posibles
                print("\n\t","Escriba la selección que desee:","\n",
                    "\n\t\t", "1. Los 10 genes más expresados en su valor medio en todas las muestras",
                    "\n\t\t", "2. Los 10 genes más expresados en su valor medio en todas las muestras WT",
                    "\n\t\t", "3. Los 10 genes más expresados en su valor medio en todas las muestras KO",
                    "\n\t\t", "4. Los 10 genes menos expresados en su valor medio en todas las muestras",
                    "\n\t\t", "5. Los 10 genes menos expresados en su valor medio en todas las muestras WT", 
                    "\n\t\t", "6. Los 10 genes menos expresados en su valor medio en todas las muestras KO",
                    "\n\n")
                
                selecion_usuario = input("\nEscriba la selección que desee.")
                
                selecion_usuario = int(error_usuario(selecion_usuario)) #filtro por si hubiese error del usuario al marcar opcion
                
                idr = inf_detallada_run(estructura_datos_runs, run, selecion_usuario)
                diccionario_genes_expresion= idr[0] #guardo diccionario con la media de expresion

                # Según eleccion usuario hago se elige más expresados o menos expresados
                if selecion_usuario <= 3 :
                    print("Los genes más expresados de", idr[2],"del", run, "son:")
                    for key, value in idr[1].items(): 
                        print("\n", key,"\n\tcon una expresion media de:\t", round(value, 3), "\n")

                if selecion_usuario >= 4 :
                    print("Los genes menos expresados de", idr[2],"del", run, "son:")
                    for key, value in idr[1].items():
                        print("\n", key,"\n\tcon una expresion media de:\t", round(value, 3), "\n")


                # Ultima elección del usuario en esta opcion para escribir el valor de corte del log fold change del RUN elegido.
                print("\n","\n","\t","Indica el valor de corte del log fold change para ver los genes más sobreexpresados y más infraexpresados.",
                      "\n","\t","Para volver al menu inicial pulsa m")
                
                selecion_usuario_lfc = input("\nEscriba el log fold change en formato numerico:")
                selecion_usuario_lfc = error_lfc(selecion_usuario_lfc) #filtro por si hubiese error del usuario al marcar el log fold change

                try:
                    float_selecion_usuario_lfc = float(selecion_usuario_lfc)
                except:
                    break
                
                # guardo el diccionario con los log fold change de cada gen
                lfc = log_fold_change(diccionario_genes_expresion, float_selecion_usuario_lfc)

                for regulacion in lfc:
                    if regulacion == "Genes sobreexpresados":
                        if len(lfc[regulacion]) == 0:   # si no hay genes sobreexpresados lanzo este print
                            print("\nNo existen genes sobreexpresados por encima del valor de log fold change elegido.")
                        else:
                            ordenado=dict(sorted(lfc[regulacion].items(), key=lambda x:x[1], reverse=True))
                            print("\n\nLos genes mas sobrexpresados del", run, "de las muestras KO con respecto a WT son:")
                            for gene, value in ordenado.items():
                                print("\n", gene,"\n\tcon un log fold change de:\t", value)
                    if regulacion == "Genes infraexpresados":
                        if len(lfc[regulacion]) == 0: # si no hay genes infraexpresados lanzo este print
                            print("\nNo existen genes infraexpresados por encima del valor de log fold change elegido.")
                        else: 
                            ordenado=dict(sorted(lfc[regulacion].items(), key=lambda x:x[1]))
                            print("\n\nLos genes mas infraexpresados de", run, "de las muestras KO con respecto a WT son:")
                            for gene, value in ordenado.items():
                                print("\n", gene,"\n\tcon un log fold change de:\t", value)
                
                #opcion para volver al menu
                while True:
                    print("\n\n\t","¿Quieres volver al menu inicial? (y/n)")
                    menu=input("\n(y/n)")
                    if menu == "y":
                        main()
                    if menu == "n":
                        break
                    break
                break   
            break
             
        elif opcion == "2":
            print("\n\n\t","Has marcado la opcion 2. Total de muestras cargadas en todos los RUN.",
                  "\n","\n","\t","Actualmente tenemos cargados los siguientes RUN:")
            for run in estructura_datos_runs:
                print("\t",run)

            igr = informacion_general(estructura_datos_runs) 
            # en [1] esta el numero de WT total, en [2] el numero de KO total, y en [3] la lista de genes totales
            
            print("\n\tEl numero total de muestras Wildtype es:", igr[1], "\n",
                  "\tEl numero total de muestras Knockout es:", igr[2], "\n", 
                  "\tEl numero de genes totales de todos los RUN es de", len(igr[3]), "diferentes")
            
             #opcion para volver al menu
            while True:
                print("\n\n\t","¿Quieres volver al menu inicial? (y/n)")
                menu=input("\n(y/n)")
                if menu == "y":
                    main()
                if menu == "n":
                    break
                break   
            break

        elif opcion == "3":
            break     

        else:
            #IMPORTANTE. Avisar al usuario cuando la entrada no es correcta
            print("\nLa opción indicada no está disponible. Pruebe con la siguientes:\n")

In [8]:
if __name__ == "__main__":
    main()
    print("\n\nEl programa ha finalizado correctamente.")



1. Informacion de los RUNs
2. Total de muestras cargadas
3. Salir del programa

 
 	 Has marcado la opcion 1. Por favor indica el RUN para ver la información general. 
 	 Para volver al menu inicial pulsa m 
 
 	 Actualmente tenemos cargados los siguientes RUN:
	 RUN1
	 RUN2
	 RUN3
	 RUN4

	 Los datos del RUN1 son:
		 El numero de muestras WT del RUN1 es:  3
		 El numero de muestras KO del RUN1 es:  4
		 Con un total de 485 genes.

	 Los datos del RUN2 son:
		 El numero de muestras WT del RUN2 es:  6
		 El numero de muestras KO del RUN2 es:  4
		 Con un total de 485 genes.

	 Los datos del RUN3 son:
		 El numero de muestras WT del RUN3 es:  4
		 El numero de muestras KO del RUN3 es:  5
		 Con un total de 485 genes.

	 Los datos del RUN4 son:
		 El numero de muestras WT del RUN4 es:  2
		 El numero de muestras KO del RUN4 es:  4
		 Con un total de 485 genes.

 
 	 Para ver más detalle de un RUN en concreto, por favor indica el RUN para ver la información detallada. 
 	 Para volver al 